In [1]:
import os
import time
import math

import pandas as pd
import tenseal as ts

In [2]:
def cleanup():
    for f in [
        "enc_ckks_perf.dat",
        "res_ckks_add.dat",
        "res_ckks_mul.dat",
        "enc_bfv_perf.dat",
        "res_bfv_add.dat",
        "res_bfv_mul.dat",
    ]:
        if os.path.exists(f):
            os.remove(f)

In [3]:
def setup_ckks():
    context = ts.context(
        ts.SCHEME_TYPE.CKKS,
        poly_modulus_degree=16384,
        coeff_mod_bit_sizes=[60, 40, 40, 40, 60],
    )
    context.global_scale = 2**40
    context.generate_galois_keys()
    return context


def setup_bfv():
    context = ts.context(
        ts.SCHEME_TYPE.BFV,
        poly_modulus_degree=16384,
        plain_modulus=1099511922689,
    )
    context.generate_galois_keys()
    context.generate_relin_keys()
    return context


def holder_encrypt_ckks(context, salary, bonus, filename):
    enc_s = ts.ckks_vector(context, salary)
    enc_b = ts.ckks_vector(context, bonus)

    ser_s = enc_s.serialize()
    ser_b = enc_b.serialize()

    with open(filename, "wb") as f:
        f.write(len(ser_s).to_bytes(4, "big"))
        f.write(ser_s)
        f.write(ser_b)


def holder_encrypt_bfv(context, salary, bonus, filename):
    enc_s = ts.bfv_vector(context, salary)
    enc_b = ts.bfv_vector(context, bonus)

    ser_s = enc_s.serialize()
    ser_b = enc_b.serialize()

    with open(filename, "wb") as f:
        f.write(len(ser_s).to_bytes(4, "big"))
        f.write(ser_s)
        f.write(ser_b)


def analyzer_process_ckks_addition(context, in_file, out_file):
    with open(in_file, "rb") as f:
        size_s = int.from_bytes(f.read(4), "big")
        bytes_s = f.read(size_s)

    enc_s = ts.ckks_vector_from(context, bytes_s)
    enc_salary_sum = enc_s.sum()

    with open(out_file, "wb") as f:
        f.write(enc_salary_sum.serialize())


def analyzer_process_ckks_multiplication(context, in_file, out_file):
    with open(in_file, "rb") as f:
        size_s = int.from_bytes(f.read(4), "big")
        bytes_s = f.read(size_s)

    enc_s = ts.ckks_vector_from(context, bytes_s)
    enc_salary_mul = enc_s.mul(2)
    enc_result = enc_salary_mul.sum()

    with open(out_file, "wb") as f:
        f.write(enc_result.serialize())


def analyzer_process_bfv_addition(context, in_file, out_file):
    with open(in_file, "rb") as f:
        size_s = int.from_bytes(f.read(4), "big")
        bytes_s = f.read(size_s)

    enc_s = ts.bfv_vector_from(context, bytes_s)
    enc_salary_sum = enc_s.sum()

    with open(out_file, "wb") as f:
        f.write(enc_salary_sum.serialize())


def analyzer_process_bfv_multiplication(context, in_file, out_file):
    with open(in_file, "rb") as f:
        size_s = int.from_bytes(f.read(4), "big")
        bytes_s = f.read(size_s)

    enc_s = ts.bfv_vector_from(context, bytes_s)
    enc_salary_mul = enc_s.mul(2)
    enc_result = enc_salary_mul.sum()

    with open(out_file, "wb") as f:
        f.write(enc_result.serialize())


def holder_decrypt_ckks_simple(context, filename):
    with open(filename, "rb") as f:
        bytes_res = f.read()
    enc_res = ts.ckks_vector_from(context, bytes_res)
    return enc_res.decrypt()[0]


def holder_decrypt_bfv_simple(context, filename):
    with open(filename, "rb") as f:
        bytes_res = f.read()
    enc_res = ts.bfv_vector_from(context, bytes_res)
    return enc_res.decrypt()[0]

In [4]:
# Read dataset
df = pd.read_csv("datasets/dataset.csv")
salaries_list = df["salary_cents"].tolist()
bonus_list = df["bonus_cents"].tolist()

print(f"Dataset Size: {len(salaries_list)} rows")

# Ground Truth
salary_sum_gt = sum(salaries_list)
salary_mul_gt = sum([s * 2 for s in salaries_list])

Dataset Size: 8192 rows


In [5]:
ctx_ckks = setup_ckks()
ctx_bfv = setup_bfv()

In [6]:
print("=" * 80)
print("PERFORMANCE COMPARISON - ADDITION OF ALL SALARIES")
print("=" * 80)

# ------------------------------------------------------------------
# Cleanup
# ------------------------------------------------------------------
cleanup()

# ==================================================================
# CKKS ADDITION
# ==================================================================
print("\n--- CKKS Addition ---")

start_total_ckks = time.time()

# Encryption
start = time.time()
holder_encrypt_ckks(ctx_ckks, salaries_list, bonus_list, "enc_ckks_perf.dat")
t_enc_ckks = time.time() - start

# Processing
start = time.time()
analyzer_process_ckks_addition(ctx_ckks, "enc_ckks_perf.dat", "res_ckks_add.dat")
t_proc_ckks = time.time() - start

# Decryption
start = time.time()
result_ckks_add = holder_decrypt_ckks_simple(ctx_ckks, "res_ckks_add.dat")
t_dec_ckks = time.time() - start

t_total_ckks_add = time.time() - start_total_ckks

print(f"Result:      {result_ckks_add:.2f}")
print(f"Encryption:  {t_enc_ckks:.4f}s")
print(f"Processing:  {t_proc_ckks:.4f}s")
print(f"Decryption:  {t_dec_ckks:.4f}s")
print(f"Total Time:  {t_total_ckks_add:.4f}s")


# ==================================================================
# BFV ADDITION
# ==================================================================
print("\n--- BFV Addition ---")

start_total_bfv = time.time()

# Encryption
start = time.time()
holder_encrypt_bfv(ctx_bfv, salaries_list, bonus_list, "enc_bfv_perf.dat")
t_enc_bfv = time.time() - start

# Processing
start = time.time()
analyzer_process_bfv_addition(ctx_bfv, "enc_bfv_perf.dat", "res_bfv_add.dat")
t_proc_bfv = time.time() - start

# Decryption
start = time.time()
result_bfv_add = holder_decrypt_bfv_simple(ctx_bfv, "res_bfv_add.dat")
t_dec_bfv = time.time() - start

t_total_bfv_add = time.time() - start_total_bfv

print(f"Result:      {result_bfv_add}")
print(f"Encryption:  {t_enc_bfv:.4f}s")
print(f"Processing:  {t_proc_bfv:.4f}s")
print(f"Decryption:  {t_dec_bfv:.4f}s")
print(f"Total Time:  {t_total_bfv_add:.4f}s\n")

PERFORMANCE COMPARISON - ADDITION OF ALL SALARIES

--- CKKS Addition ---
Result:      2244383855.00
Encryption:  0.0416s
Processing:  0.1129s
Decryption:  0.0151s
Total Time:  0.1698s

--- BFV Addition ---
Result:      2244383855
Encryption:  0.0486s
Processing:  0.2848s
Decryption:  0.0205s
Total Time:  0.3541s



In [8]:
print("=" * 80)
print("PERFORMANCE COMPARISON - MULTIPLICATION OF ALL SALARIES BY 2")
print("=" * 80)

# ------------------------------------------------------------------
# Cleanup
# ------------------------------------------------------------------
cleanup()

# ==================================================================
# CKKS Multiplication
# ==================================================================
print("\n--- CKKS Multiplication ---")

start_total_ckks_mul = time.time()

# Encryption
start = time.time()
holder_encrypt_ckks(ctx_ckks, salaries_list, bonus_list, "enc_ckks_perf.dat")
t_enc_ckks_mul = time.time() - start

# Processing
start = time.time()
analyzer_process_ckks_multiplication(ctx_ckks, "enc_ckks_perf.dat", "res_ckks_mul.dat")
t_proc_ckks_mul = time.time() - start

# Decryption
start = time.time()
result_ckks_mul = holder_decrypt_ckks_simple(ctx_ckks, "res_ckks_mul.dat")
t_dec_ckks_mul = time.time() - start

t_total_ckks_mul = time.time() - start_total_ckks_mul

print(f"Result: {result_ckks_mul:.2f}")
print(f"Encryption:  {t_enc_ckks_mul:.4f}s")
print(f"Processing:  {t_proc_ckks_mul:.4f}s")
print(f"Decryption:  {t_dec_ckks_mul:.4f}s")
print(f"Total Time:  {t_total_ckks_mul:.4f}s")

# ==================================================================
# BFV Multiplication
# ==================================================================
print("\n--- BFV Multiplication ---")

start_total_bfv_mul = time.time()

# Encryption
start = time.time()
holder_encrypt_bfv(ctx_bfv, salaries_list, bonus_list, "enc_bfv_perf.dat")
t_enc_bfv_mul = time.time() - start

# Processing
start = time.time()
analyzer_process_bfv_multiplication(ctx_bfv, "enc_bfv_perf.dat", "res_bfv_mul.dat")
t_proc_bfv_mul = time.time() - start

# Decryption
start = time.time()
result_bfv_mul = holder_decrypt_bfv_simple(ctx_bfv, "res_bfv_mul.dat")
t_dec_bfv_mul = time.time() - start

t_total_bfv_mul = time.time() - start_total_bfv_mul

print(f"Result: {result_bfv_mul}")
print(f"Encryption:  {t_enc_bfv_mul:.4f}s")
print(f"Processing:  {t_proc_bfv_mul:.4f}s")
print(f"Decryption:  {t_dec_bfv_mul:.4f}s")
print(f"Total Time:  {t_total_bfv_mul:.4f}s")


PERFORMANCE COMPARISON - MULTIPLICATION OF ALL SALARIES BY 2

--- CKKS Multiplication ---
Result: 4488774131.24
Encryption:  0.0494s
Processing:  0.0961s
Decryption:  0.0148s
Total Time:  0.1604s

--- BFV Multiplication ---
Result: 4488767710
Encryption:  0.0496s
Processing:  0.3051s
Decryption:  0.0213s
Total Time:  0.3762s


In [9]:
print("\n--- Performance Verification ---")
print(f"\nAddition (Sum of Salaries):")
print(f"  Ground Truth: {salary_sum_gt}")
print(f"  CKKS Result:  {result_ckks_add:.2f}")
print(f"  BFV Result:   {result_bfv_add}")
print(f"  CKKS Error:   {abs(result_ckks_add - salary_sum_gt):.4f}")
print(f"  BFV Error:    {abs(result_bfv_add - salary_sum_gt)}")

print(f"\nMultiplication (Salaries * 2):")
print(f"  Ground Truth: {salary_mul_gt}")
print(f"  CKKS Result:  {result_ckks_mul:.2f}")
print(f"  BFV Result:   {result_bfv_mul}")
print(f"  CKKS Error:   {abs(result_ckks_mul - salary_mul_gt):.4f}")
print(f"  BFV Error:    {abs(result_bfv_mul - salary_mul_gt)}")

print("\n--- Performance Summary ---")
print(f"\nCKKS vs BFV - Addition:")
print(f"  CKKS Total Time: {t_total_ckks_add:.4f}s")
print(f"  BFV Total Time:  {t_total_bfv_add:.4f}s")
print(f"  Speedup:         {t_total_ckks_add / t_total_bfv_add:.2f}x {'(BFV faster)' if t_total_bfv_add < t_total_ckks_add else '(CKKS faster)'}")

print(f"\nCKKS vs BFV - Multiplication:")
print(f"  CKKS Total Time: {t_total_ckks_mul:.4f}s")
print(f"  BFV Total Time:  {t_total_bfv_mul:.4f}s")
print(f"  Speedup:         {t_total_ckks_mul / t_total_bfv_mul:.2f}x {'(BFV faster)' if t_total_bfv_mul < t_total_ckks_mul else '(CKKS faster)'}")


--- Performance Verification ---

Addition (Sum of Salaries):
  Ground Truth: 2244383855
  CKKS Result:  2244383855.00
  BFV Result:   2244383855
  CKKS Error:   0.0000
  BFV Error:    0

Multiplication (Salaries * 2):
  Ground Truth: 4488767710
  CKKS Result:  4488774131.24
  BFV Result:   4488767710
  CKKS Error:   6421.2388
  BFV Error:    0

--- Performance Summary ---

CKKS vs BFV - Addition:
  CKKS Total Time: 0.1698s
  BFV Total Time:  0.3541s
  Speedup:         0.48x (CKKS faster)

CKKS vs BFV - Multiplication:
  CKKS Total Time: 0.1604s
  BFV Total Time:  0.3762s
  Speedup:         0.43x (CKKS faster)
